### Understanding docker first run

- docker run -t homework:1 .
- docker run -it homework:1
- pip 24.3.1 from /usr/local/lib/python3.12/site-packages/pip (python 3.12)

### Understanding Docker networking and docker-compose

- Host: postgres (the container name)
- Port: 5432
- Database: ny_taxi

docker run -it  
--network=de_zoomcamp2025_default  
data_ingest:taxi     
--user=postgres     
--password=postgres     
--host=postgres     
--port=5432     
--db=ny_taxi     
--table_name=green_taxi_trips     
--url="$URL"

docker run -it  
--network=de_zoomcamp2025_default  
data_ingest:taxi     
--user=postgres     
--password=postgres     
--host=postgres     
--port=5432     
--db=ny_taxi     
--table_name=taxi_zone_lookup     
--url="wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"

### Trip Segmentation Count

```
--Up to 1 mile
SELECT COUNT(*)
FROM green_taxi_trips AS gt
WHERE gt.lpep_dropoff_datetime >= '2019-10-01'
  AND gt.lpep_dropoff_datetime < '2019-11-01'
  AND gt.trip_distance <= 1;


--In between 1 (exclusive) and 3 miles (inclusive)
SELECT COUNT(*)
FROM green_taxi_trips AS gt
WHERE gt.lpep_dropoff_datetime >= '2019-10-01'
  AND gt.lpep_dropoff_datetime < '2019-11-01'
  AND gt.trip_distance > 1
  AND gt.trip_distance <= 3;
  
--In between 3 (exclusive) and 7 miles (inclusive),
SELECT COUNT(*)
FROM green_taxi_trips AS gt
WHERE gt.lpep_dropoff_datetime >= '2019-10-01'
  AND gt.lpep_dropoff_datetime < '2019-11-01'
  AND gt.trip_distance > 3
  AND gt.trip_distance <= 7;
  
--In between 7 (exclusive) and 10 miles (inclusive),
SELECT COUNT(*)
FROM green_taxi_trips AS gt
WHERE gt.lpep_dropoff_datetime >= '2019-10-01'
  AND gt.lpep_dropoff_datetime < '2019-11-01'
  AND gt.trip_distance > 7
  AND gt.trip_distance <= 10;


--Over 10 miles
SELECT COUNT(*)
FROM green_taxi_trips AS gt
WHERE gt.lpep_dropoff_datetime >= '2019-10-01'
  AND gt.lpep_dropoff_datetime < '2019-11-01'
  AND gt.trip_distance > 10;
```

### Longest trip for each day

```
SELECT gt.lpep_pickup_datetime
FROM green_taxi_trips AS gt
WHERE gt.trip_distance = (
    SELECT MAX(trip_distance) 
    FROM green_taxi_trips
);
```
- 2019-10-31 23:23:41

### Three biggest pickup zones

```
SELECT tz."Zone", 
       SUM(gt.total_amount) AS total
FROM green_taxi_trips AS gt
LEFT JOIN taxi_zone_lookup AS tz
    ON gt."PULocationID" = tz."LocationID"
WHERE DATE(gt.lpep_pickup_datetime) = '2019-10-18'
GROUP BY tz."Zone"
HAVING SUM(gt.total_amount) > 13000
ORDER BY total DESC;
```

### Largest tip

```
SELECT tzd."Zone" as dropoff_loc,
       MAX(gt.tip_amount) as max_tip
FROM green_taxi_trips AS gt
LEFT JOIN taxi_zone_lookup tzp 
    ON gt."PULocationID" = tzp."LocationID"
JOIN taxi_zone_lookup tzd 
    ON gt."DOLocationID" = tzd."LocationID"
WHERE DATE(gt.lpep_pickup_datetime) = '2019-10-19'
  AND tzp."Zone" = 'East Harlem North'
GROUP BY tzd."Zone"
ORDER BY max_tip DESC
LIMIT 5;
```

### Terraform Workflow